In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
mySchema = StructType([StructField("Player_Name",StringType(),True),
                       StructField("Stadium",StringType(),True),
                       StructField("Runs",IntegerType(),True),
                       StructField("Wickets",IntegerType(),True),
                       StructField("Catches",IntegerType(),True)
                       ])

data = [("Jadeja","Wankhede",40,2,1),
        ("Hardik","Eden Garden",55,0,2),
        ("Jadeja","Eden Garden",25,3,0),
        ("Watson","Wankhede",30,1,0),
        ("Hardik","Wankhede",45,0,1),
        ("Jadeja","Wankhede",60,1,1),
        ("Hardik","Eden Garden",45,0,3),
        ("Hardik","Eden Garden",30,4,0),
        ("Watson","Wankhede",20,2,2),
        ("Watson","Eden Garden",50,0,0)]

df = spark.createDataFrame(data,mySchema)

df.show()

Lets Group Data According To Player_Name and PIVOT on Stadium

In [0]:
pivot_df = df.groupBy("Player_Name").pivot("Stadium").sum("Runs")
pivot_df.show()

Lets apply Sum on 2 Columns

In [0]:
pivot_df = df.groupBy("Player_Name").pivot("Stadium").sum("Runs","Wickets")
pivot_df.show()

Adding Rename to Upper Query

In [0]:
pivot_df2 = pivot_df.withColumnRenamed("Eden Garden_sum(Runs)","Eden_Garden(Runs)")\
  .withColumnRenamed("Eden Garden_sum(Wickets)","Eden_Garden(Witckets)")\
    .withColumnRenamed("Wankhede_sum(Runs)","Wankhede(Runs)")\
    .withColumnRenamed("Wankhede_sum(Wickets)","Wankhede(Wickets)")

pivot_df2.show()

After Pivoting On Stadium Column, We want only "Wankhede" Stadium column to be pivoted 

In [0]:
df_pivot = df.groupBy("Player_Name").pivot("Stadium",['Wankhede']).sum("Runs")
df_pivot.show()

# AND

df_pivot = df.groupBy("Player_Name").pivot("Stadium",['Wankhede']).sum("Runs","Wickets")
df_pivot.show()

### UN-PIVOT
- There is no function as "Un-Pivot"
- We Do "Un-Pivot" using Stack_expression

Raw Data 

In [0]:
pivot_df = df.groupBy("Player_Name").pivot("Stadium").sum("Runs")
pivot_df = pivot_df.withColumnRenamed("Eden Garden","Eden_Garden")
pivot_df.show()

Crating Stack Expression

In [0]:
stack_expr = "stack(1,'Eden_Garden',Eden_Garden)"

Using Stack Expression to Un-Pivot ONLY Eden_Garden Column Data

In [0]:
unPivot_df = pivot_df.select("Player_Name",expr(stack_expr))
unPivot_df.show()

Un-Pivot Stadium (for Both 'Wankhede' and 'Eden_Garden')

In [0]:
stack_expr = "stack(2,'Eden_Garden',Eden_Garden,'Wankhede',Wankhede)"

In [0]:
unPivot_df = pivot_df.select("Player_Name",expr(stack_expr))
unPivot_df.show()

In [0]:
unPivot_df = unPivot_df.withColumnRenamed("col0","Stadium").withColumnRenamed("col1","Total_Runs")
unPivot_df.show()